# Baseline Experiment: bbox3 + fix_direction

This notebook implements the baseline approach from top kernels:
1. Use bbox3 binary optimizer
2. Apply fix_direction rotation optimization
3. Validate and score

In [1]:
import shutil
import os

# Setup working directory
os.chdir('/home/code/experiments/001_baseline')

# Copy bbox3 binary and starting submission
shutil.copy('/home/code/datasets/bucket-of-chump/bbox3', './bbox3')
shutil.copy('/home/code/datasets/santa-2025-csv/santa-2025.csv', './submission.csv')

print('Files copied successfully')
print('Starting submission:', os.path.getsize('./submission.csv'), 'bytes')

Files copied successfully
Starting submission: 1525109 bytes


In [2]:
# Write tree_packer_v21.cpp
tree_packer_cpp = '''
// Tree Packer v21 - ENHANCED v19 with SWAP MOVES + MULTI-START
// Compile: g++ -O3 -march=native -std=c++17 -fopenmp -o tree_packer_v21 tree_packer_v21.cpp

#include <bits/stdc++.h>
#include <omp.h>
using namespace std;

constexpr int MAX_N = 200;
constexpr int NV = 15;
constexpr double PI = 3.14159265358979323846;

alignas(64) const long double TX[NV] = {0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125};
alignas(64) const long double TY[NV] = {0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5};

struct FastRNG {
    uint64_t s[2];
    FastRNG(uint64_t seed = 42) {
        s[0] = seed ^ 0x853c49e6748fea9bULL;
        s[1] = (seed * 0x9e3779b97f4a7c15ULL) ^ 0xc4ceb9fe1a85ec53ULL;
    }
    inline uint64_t rotl(uint64_t x, int k) { return (x << k) | (x >> (64 - k)); }
    inline uint64_t next() {
        uint64_t s0 = s[0], s1 = s[1], r = s0 + s1;
        s1 ^= s0; s[0] = rotl(s0, 24) ^ s1 ^ (s1 << 16); s[1] = rotl(s1, 37);
        return r;
    }
    inline long double rf() { return (next() >> 11) * 0x1.0p-53L; }
    inline long double rf2() { return rf() * 2.0L - 1.0L; }
    inline int ri(int n) { return next() % n; }
    inline long double gaussian() {
        long double u1 = rf() + 1e-10L, u2 = rf();
        return sqrtl(-2.0L * logl(u1)) * cosl(2.0L * PI * u2);
    }
};

struct Poly {
    long double px[NV], py[NV];
    long double x0, y0, x1, y1;
};

inline void getPoly(long double cx, long double cy, long double deg, Poly& q) {
    long double rad = deg * (PI / 180.0L);
    long double s = sinl(rad), c = cosl(rad);
    long double minx = 1e9L, miny = 1e9L, maxx = -1e9L, maxy = -1e9L;
    for (int i = 0; i < NV; i++) {
        long double x = TX[i] * c - TY[i] * s + cx;
        long double y = TX[i] * s + TY[i] * c + cy;
        q.px[i] = x; q.py[i] = y;
        if (x < minx) minx = x; if (x > maxx) maxx = x;
        if (y < miny) miny = y; if (y > maxy) maxy = y;
    }
    q.x0 = minx; q.y0 = miny; q.x1 = maxx; q.y1 = maxy;
}

inline bool pip(long double px, long double py, const Poly& q) {
    bool in = false;
    int j = NV - 1;
    for (int i = 0; i < NV; i++) {
        if ((q.py[i] > py) != (q.py[j] > py) &&
            px < (q.px[j] - q.px[i]) * (py - q.py[i]) / (q.py[j] - q.py[i]) + q.px[i])
            in = !in;
        j = i;
    }
    return in;
}

inline bool segInt(long double ax, long double ay, long double bx, long double by,
                   long double cx, long double cy, long double dx, long double dy) {
    long double d1 = (dx-cx)*(ay-cy) - (dy-cy)*(ax-cx);
    long double d2 = (dx-cx)*(by-cy) - (dy-cy)*(bx-cx);
    long double d3 = (bx-ax)*(cy-ay) - (by-ay)*(cx-ax);
    long double d4 = (bx-ax)*(dy-ay) - (by-ay)*(dx-ax);
    return ((d1 > 0) != (d2 > 0)) && ((d3 > 0) != (d4 > 0));
}

inline bool overlap(const Poly& a, const Poly& b) {
    if (a.x1 < b.x0 || b.x1 < a.x0 || a.y1 < b.y0 || b.y1 < a.y0) return false;
    for (int i = 0; i < NV; i++) {
        if (pip(a.px[i], a.py[i], b)) return true;
        if (pip(b.px[i], b.py[i], a)) return true;
    }
    for (int i = 0; i < NV; i++) {
        int ni = (i + 1) % NV;
        for (int j = 0; j < NV; j++) {
            int nj = (j + 1) % NV;
            if (segInt(a.px[i], a.py[i], a.px[ni], a.py[ni],
                      b.px[j], b.py[j], b.px[nj], b.py[nj])) return true;
        }
    }
    return false;
}

struct Cfg {
    int n;
    long double x[MAX_N], y[MAX_N], a[MAX_N];
    Poly pl[MAX_N];
    long double gx0, gy0, gx1, gy1;

    inline void upd(int i) { getPoly(x[i], y[i], a[i], pl[i]); }
    inline void updAll() { for (int i = 0; i < n; i++) upd(i); updGlobal(); }

    inline void updGlobal() {
        gx0 = gy0 = 1e9L; gx1 = gy1 = -1e9L;
        for (int i = 0; i < n; i++) {
            if (pl[i].x0 < gx0) gx0 = pl[i].x0;
            if (pl[i].x1 > gx1) gx1 = pl[i].x1;
            if (pl[i].y0 < gy0) gy0 = pl[i].y0;
            if (pl[i].y1 > gy1) gy1 = pl[i].y1;
        }
    }

    inline bool hasOvl(int i) const {
        for (int j = 0; j < n; j++)
            if (i != j && overlap(pl[i], pl[j])) return true;
        return false;
    }

    inline bool anyOvl() const {
        for (int i = 0; i < n; i++)
            for (int j = i + 1; j < n; j++)
                if (overlap(pl[i], pl[j])) return true;
        return false;
    }

    inline long double side() const { return max(gx1 - gx0, gy1 - gy0); }
    inline long double score() const { long double s = side(); return s * s / n; }
};

// Squeeze
Cfg squeeze(Cfg c) {
    long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
    for (long double scale = 0.9995L; scale >= 0.98L; scale -= 0.0005L) {
        Cfg trial = c;
        for (int i = 0; i < c.n; i++) {
            trial.x[i] = cx + (c.x[i] - cx) * scale;
            trial.y[i] = cy + (c.y[i] - cy) * scale;
        }
        trial.updAll();
        if (!trial.anyOvl()) c = trial;
        else break;
    }
    return c;
}

// Compaction
Cfg compaction(Cfg c, int iters) {
    long double bs = c.side();
    for (int it = 0; it < iters; it++) {
        long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
        bool improved = false;
        for (int i = 0; i < c.n; i++) {
            long double ox = c.x[i], oy = c.y[i];
            long double dx = cx - c.x[i], dy = cy - c.y[i];
            long double d = sqrtl(dx*dx + dy*dy);
            if (d < 1e-6L) continue;
            for (long double step : {0.02L, 0.008L, 0.003L, 0.001L, 0.0004L}) {
                c.x[i] = ox + dx/d * step; c.y[i] = oy + dy/d * step; c.upd(i);
                if (!c.hasOvl(i)) {
                    c.updGlobal();
                    if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; ox = c.x[i]; oy = c.y[i]; }
                    else { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
                } else { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
            }
        }
        c.updGlobal();
        if (!improved) break;
    }
    return c;
}

// Local search
Cfg localSearch(Cfg c, int maxIter) {
    long double bs = c.side();
    const long double steps[] = {0.01L, 0.004L, 0.0015L, 0.0006L, 0.00025L, 0.0001L};
    const long double rots[] = {5.0L, 2.0L, 0.8L, 0.3L, 0.1L};
    const int dx[] = {1,-1,0,0,1,1,-1,-1};
    const int dy[] = {0,0,1,-1,1,-1,1,-1};

    for (int iter = 0; iter < maxIter; iter++) {
        bool improved = false;
        for (int i = 0; i < c.n; i++) {
            long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
            long double ddx = cx - c.x[i], ddy = cy - c.y[i];
            long double dist = sqrtl(ddx*ddx + ddy*ddy);
            if (dist > 1e-6L) {
                for (long double st : steps) {
                    long double ox = c.x[i], oy = c.y[i];
                    c.x[i] += ddx/dist * st; c.y[i] += ddy/dist * st; c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; }
                        else { c.x[i]=ox; c.y[i]=oy; c.upd(i); c.updGlobal(); } }
                    else { c.x[i]=ox; c.y[i]=oy; c.upd(i); }
                }
            }
            for (long double st : steps) {
                for (int d = 0; d < 8; d++) {
                    long double ox=c.x[i], oy=c.y[i];
                    c.x[i] += dx[d]*st; c.y[i] += dy[d]*st; c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; }
                        else { c.x[i]=ox; c.y[i]=oy; c.upd(i); c.updGlobal(); } }
                    else { c.x[i]=ox; c.y[i]=oy; c.upd(i); }
                }
            }
            for (long double rt : rots) {
                for (long double da : {rt, -rt}) {
                    long double oa = c.a[i]; c.a[i] += da;
                    while (c.a[i] < 0) c.a[i] += 360; while (c.a[i] >= 360) c.a[i] -= 360;
                    c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; }
                        else { c.a[i]=oa; c.upd(i); c.updGlobal(); } }
                    else { c.a[i]=oa; c.upd(i); }
                }
            }
        }
        if (!improved) break;
    }
    return c;
}

// Simulated Annealing with swap moves
Cfg saWithSwap(Cfg c, int iters, FastRNG& rng, long double T0 = 0.05L) {
    long double bs = c.side();
    Cfg best = c;
    long double T = T0;
    long double alpha = powl(0.0001L / T0, 1.0L / iters);
    
    for (int it = 0; it < iters; it++) {
        int moveType = rng.ri(4);
        
        if (moveType == 0) { // Translation
            int i = rng.ri(c.n);
            long double ox = c.x[i], oy = c.y[i];
            long double step = T * 0.5L;
            c.x[i] += rng.gaussian() * step;
            c.y[i] += rng.gaussian() * step;
            c.upd(i);
            if (c.hasOvl(i)) { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
            else {
                c.updGlobal();
                long double ns = c.side();
                if (ns < bs) { bs = ns; best = c; }
                else if (rng.rf() > expl(-(ns - bs) / T)) { c.x[i] = ox; c.y[i] = oy; c.upd(i); c.updGlobal(); }
            }
        } else if (moveType == 1) { // Rotation
            int i = rng.ri(c.n);
            long double oa = c.a[i];
            c.a[i] += rng.gaussian() * T * 30.0L;
            while (c.a[i] < 0) c.a[i] += 360; while (c.a[i] >= 360) c.a[i] -= 360;
            c.upd(i);
            if (c.hasOvl(i)) { c.a[i] = oa; c.upd(i); }
            else {
                c.updGlobal();
                long double ns = c.side();
                if (ns < bs) { bs = ns; best = c; }
                else if (rng.rf() > expl(-(ns - bs) / T)) { c.a[i] = oa; c.upd(i); c.updGlobal(); }
            }
        } else if (moveType == 2 && c.n >= 2) { // Swap positions
            int i = rng.ri(c.n), j = rng.ri(c.n);
            if (i != j) {
                swap(c.x[i], c.x[j]); swap(c.y[i], c.y[j]);
                c.upd(i); c.upd(j);
                if (c.hasOvl(i) || c.hasOvl(j)) {
                    swap(c.x[i], c.x[j]); swap(c.y[i], c.y[j]);
                    c.upd(i); c.upd(j);
                } else {
                    c.updGlobal();
                    long double ns = c.side();
                    if (ns < bs) { bs = ns; best = c; }
                    else if (rng.rf() > expl(-(ns - bs) / T)) {
                        swap(c.x[i], c.x[j]); swap(c.y[i], c.y[j]);
                        c.upd(i); c.upd(j); c.updGlobal();
                    }
                }
            }
        } else { // Move towards center
            int i = rng.ri(c.n);
            long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
            long double ox = c.x[i], oy = c.y[i];
            long double dx = cx - c.x[i], dy = cy - c.y[i];
            long double d = sqrtl(dx*dx + dy*dy);
            if (d > 1e-6L) {
                long double step = T * 0.3L;
                c.x[i] += dx/d * step; c.y[i] += dy/d * step;
                c.upd(i);
                if (c.hasOvl(i)) { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
                else {
                    c.updGlobal();
                    long double ns = c.side();
                    if (ns < bs) { bs = ns; best = c; }
                    else if (rng.rf() > expl(-(ns - bs) / T)) { c.x[i] = ox; c.y[i] = oy; c.upd(i); c.updGlobal(); }
                }
            }
        }
        T *= alpha;
    }
    return best;
}

Cfg configs[MAX_N + 1];
long double best_sides[MAX_N + 1];

void parse_csv(const string& fn) {
    ifstream f(fn);
    string line;
    getline(f, line); // header
    
    map<int, vector<tuple<int, long double, long double, long double>>> data;
    
    while (getline(f, line)) {
        // Parse: id,x,y,deg where values have 's' prefix
        size_t p1 = line.find(',');
        size_t p2 = line.find(',', p1+1);
        size_t p3 = line.find(',', p2+1);
        
        string id = line.substr(0, p1);
        string xs = line.substr(p1+2, p2-p1-2); // skip 's'
        string ys = line.substr(p2+2, p3-p2-2);
        string ds = line.substr(p3+2);
        
        size_t us = id.find('_');
        int n = stoi(id.substr(0, us));
        int idx = stoi(id.substr(us+1));
        
        data[n].push_back({idx, stold(xs), stold(ys), stold(ds)});
    }
    
    for (auto& [n, trees] : data) {
        sort(trees.begin(), trees.end());
        configs[n].n = n;
        for (int i = 0; i < n; i++) {
            configs[n].x[i] = get<1>(trees[i]);
            configs[n].y[i] = get<2>(trees[i]);
            configs[n].a[i] = get<3>(trees[i]);
        }
        configs[n].updAll();
        best_sides[n] = configs[n].side();
    }
}

void save_csv(const string& fn) {
    ofstream f(fn);
    f << fixed << setprecision(15);
    f << "id,x,y,deg\\n";
    for (int n = 1; n <= MAX_N; n++) {
        for (int i = 0; i < n; i++) {
            f << setfill('0') << setw(3) << n << "_" << i << ",s" 
              << configs[n].x[i] << ",s" << configs[n].y[i] << ",s" << configs[n].a[i] << "\\n";
        }
    }
}

long double calc_total_score() {
    long double total = 0;
    for (int n = 1; n <= MAX_N; n++) {
        long double s = best_sides[n];
        total += s * s / n;
    }
    return total;
}

int main(int argc, char** argv) {
    int n_iters = 5000;
    int restarts = 16;
    
    for (int i = 1; i < argc; i++) {
        if (string(argv[i]) == "-n" && i+1 < argc) n_iters = stoi(argv[++i]);
        if (string(argv[i]) == "-r" && i+1 < argc) restarts = stoi(argv[++i]);
    }
    
    parse_csv("submission.csv");
    
    cout << fixed << setprecision(8);
    cout << "Initial: " << calc_total_score() << "\\n";
    
    #pragma omp parallel for schedule(dynamic)
    for (int n = 1; n <= MAX_N; n++) {
        FastRNG rng(n * 12345 + 67890);
        Cfg best = configs[n];
        long double bs = best.side();
        
        for (int r = 0; r < restarts; r++) {
            Cfg c = configs[n];
            
            // Multi-angle restart
            if (r > 0) {
                long double angle = rng.rf() * 360.0L;
                long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
                long double rad = angle * PI / 180.0L;
                long double cs = cosl(rad), sn = sinl(rad);
                for (int i = 0; i < c.n; i++) {
                    long double dx = c.x[i] - cx, dy = c.y[i] - cy;
                    c.x[i] = cx + dx * cs - dy * sn;
                    c.y[i] = cy + dx * sn + dy * cs;
                    c.a[i] += angle;
                    while (c.a[i] >= 360) c.a[i] -= 360;
                }
                c.updAll();
            }
            
            c = squeeze(c);
            c = compaction(c, 20);
            c = localSearch(c, 10);
            c = saWithSwap(c, n_iters, rng);
            c = localSearch(c, 20);
            c = squeeze(c);
            c = compaction(c, 30);
            
            if (c.side() < bs) {
                bs = c.side();
                best = c;
            }
        }
        
        #pragma omp critical
        {
            if (bs < best_sides[n]) {
                configs[n] = best;
                best_sides[n] = bs;
            }
        }
    }
    
    cout << "Final:   " << calc_total_score() << "\\n";
    save_csv("submission_v21.csv");
    
    return 0;
}
'''

with open('tree_packer_v21.cpp', 'w') as f:
    f.write(tree_packer_cpp)
print('tree_packer_v21.cpp written')

./bbox3: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by ./bbox3)



In [3]:
# Compile tree_packer_v21
import subprocess
import os

os.environ['OMP_NUM_THREADS'] = '96'

result = subprocess.run(
    ['g++', '-O3', '-march=native', '-std=c++17', '-fopenmp', '-o', 'tree_packer_v21', 'tree_packer_v21.cpp'],
    capture_output=True, text=True
)
if result.returncode == 0:
    print('Compilation successful!')
else:
    print('Compilation failed:')
    print(result.stderr)

bbox3 completed in 0.0 seconds
Output: 
Errors: ./bbox3: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by ./bbox3)



In [ ]:
# Core classes and functions for scoring and validation
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from shapely.ops import unary_union
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar

getcontext().prec = 30
scale_factor = Decimal('1e18')

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated, 
            xoff=float(self.center_x * scale_factor), 
            yoff=float(self.center_y * scale_factor)
        )

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print('ChristmasTree class defined')

In [ ]:
def load_configuration_from_df(n, df):
    """Loads all trees for a given N from the submission DataFrame."""
    group_data = df[df['id'].str.startswith(f'{n:03d}_')]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row['x'])[1:] if str(row['x']).startswith('s') else str(row['x'])
        y = str(row['y'])[1:] if str(row['y']).startswith('s') else str(row['y'])
        deg = str(row['deg'])[1:] if str(row['deg']).startswith('s') else str(row['deg'])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_tree_list_side_length(tree_list):
    """Get the side length of the bounding square."""
    if not tree_list:
        return Decimal('0')
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1]))) / scale_factor

def get_score(trees, n):
    """Calculates the score (S^2 / N) for a given configuration."""
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side_length = max(max_x - min_x, max_y - min_y)
    return side_length**2 / n

def has_overlap(trees):
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print('Helper functions defined')

In [ ]:
def score_and_validate_submission(file_path, max_n=200):
    """Reads a submission CSV, calculates the total score, and checks for overlaps."""
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f'Error reading CSV: {e}')
        return {'status': 'FAILED', 'error': str(e)}

    total_score = 0.0
    failed_overlap_n = []
    
    print(f'--- Scoring and Validation: {file_path} (N=1 to {max_n}) ---')

    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
                print(f'  ❌ N={n:03d}: OVERLAP DETECTED! (Score: {current_score:.6f})')
        
    print('\n--- Summary ---')
    if failed_overlap_n:
        print(f'❌ Validation FAILED: Overlaps found in N: {failed_overlap_n}')
        status = 'FAILED (Overlaps)'
    else:
        print('✅ Validation SUCCESSFUL: No overlaps detected.')
        status = 'SUCCESS'
        
    print(f'**Total Score (Σ S²/N): {total_score:.10f}**')
    
    return {
        'status': status,
        'total_score': total_score,
        'failed_overlap_n': failed_overlap_n
    }

print('Validation function defined')

In [ ]:
# Score the submission after bbox3 optimization
result_after_bbox3 = score_and_validate_submission('submission.csv', max_n=200)
print(f"\nScore after bbox3: {result_after_bbox3['total_score']:.10f}")

In [ ]:
# fix_direction functions
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)
    hull_points = points_np[ConvexHull(points_np).vertices]
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999), method='bounded'
    )
    found_angle_deg = res.x
    found_side = res.fun
    
    improvement = initial_side - found_side
    EPSILON = 1e-8
    
    if improvement > EPSILON:
        return Decimal(str(found_side)) / scale_factor, found_angle_deg
    else:
        return Decimal(str(initial_side)) / scale_factor, 0.0

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center / float(scale_factor)
    rotated = shifted.dot(rot_matrix.T) + rotation_center / float(scale_factor)
    
    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print('fix_direction functions defined')

In [ ]:
def fix_direction(input_path='submission.csv', output_path='submission.csv', passes=2):
    """Apply rotation optimization to all configurations."""
    df = pd.read_csv(input_path)
    
    # Parse all groups
    df['x_val'] = df['x'].str.strip('s')
    df['y_val'] = df['y'].str.strip('s')
    df['deg_val'] = df['deg'].str.strip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    dict_of_tree_list = {}
    dict_of_side_length = {}
    
    for group_id, group_data in df.groupby('group_id'):
        tree_list = [ChristmasTree(row['x_val'], row['y_val'], row['deg_val'])
                     for _, row in group_data.iterrows()]
        dict_of_tree_list[group_id] = tree_list
        dict_of_side_length[group_id] = get_tree_list_side_length(tree_list)
    
    # Calculate initial score
    initial_score = sum(v**2 / Decimal(k) for k, v in dict_of_side_length.items())
    print(f'Initial score: {float(initial_score):.10f}')
    
    # Apply fix_direction for multiple passes
    for pass_num in range(passes):
        improvements = 0
        for group_id in sorted(dict_of_tree_list.keys()):
            trees = dict_of_tree_list[group_id]
            if len(trees) <= 1:
                continue
            
            best_side, best_angle = optimize_rotation(trees)
            
            if best_angle > 0.001:
                rotated_trees = apply_rotation(trees, best_angle)
                new_side = get_tree_list_side_length(rotated_trees)
                
                if new_side < dict_of_side_length[group_id]:
                    dict_of_tree_list[group_id] = rotated_trees
                    dict_of_side_length[group_id] = new_side
                    improvements += 1
        
        current_score = sum(v**2 / Decimal(k) for k, v in dict_of_side_length.items())
        print(f'Pass {pass_num + 1}: {improvements} improvements, score: {float(current_score):.10f}')
    
    # Save results
    rows = []
    for group_id in sorted(dict_of_tree_list.keys(), key=lambda x: int(x)):
        for i, tree in enumerate(dict_of_tree_list[group_id]):
            rows.append({
                'id': f'{group_id}_{i}',
                'x': f's{tree.center_x}',
                'y': f's{tree.center_y}',
                'deg': f's{tree.angle}'
            })
    
    result_df = pd.DataFrame(rows)
    result_df.to_csv(output_path, index=False)
    
    final_score = sum(v**2 / Decimal(k) for k, v in dict_of_side_length.items())
    print(f'\nFinal score after fix_direction: {float(final_score):.10f}')
    return float(final_score)

print('fix_direction main function defined')

In [ ]:
# Apply fix_direction with 2 passes
final_score = fix_direction('submission.csv', 'submission.csv', passes=2)

In [ ]:
# Final validation
final_result = score_and_validate_submission('submission.csv', max_n=200)
print(f"\nFinal validated score: {final_result['total_score']:.10f}")

In [ ]:
# Replace any overlapping configurations with donor file
def replace_invalid_configurations(new_csv_path, good_csv_path, output_csv_path, failed_n_list):
    if not failed_n_list:
        print('No overlaps to fix')
        return
    
    df_new = pd.read_csv(new_csv_path)
    df_good = pd.read_csv(good_csv_path)
    failed_prefixes = [f'{n:03d}_' for n in failed_n_list]
    df_to_keep = df_new[~df_new['id'].str.startswith(tuple(failed_prefixes))]
    df_replacement = df_good[df_good['id'].str.startswith(tuple(failed_prefixes))]
    df_repaired = pd.concat([df_to_keep, df_replacement]).sort_values(by='id').reset_index(drop=True)
    df_repaired.to_csv(output_csv_path, index=False)
    print(f'Replaced {len(failed_n_list)} configurations with donor file')

if final_result['failed_overlap_n']:
    replace_invalid_configurations(
        'submission.csv',
        '/home/code/datasets/santa-2025-csv/santa-2025.csv',
        'submission.csv',
        final_result['failed_overlap_n']
    )
    # Re-validate
    final_result = score_and_validate_submission('submission.csv', max_n=200)
    print(f"\nScore after repair: {final_result['total_score']:.10f}")

In [ ]:
# Copy to submission folder
import shutil
shutil.copy('submission.csv', '/home/submission/submission.csv')
print(f"Final submission saved to /home/submission/submission.csv")
print(f"Final score: {final_result['total_score']:.10f}")